# 公開データを整理する
公開するデータを整理するタスクです。<br>
公開したいデータを選択し、整理を行って下さい。


## 公開データを整理する
以下の手順に従って公開対象となるデータの整理を行ってください。

### 1. リサーチフローデータをコピーする
このサブフローの親となっている実験サブフローと査読サブフローからデータをコピーします。<br>
査読サブフローが存在しない場合は論文執筆サブフローからデータをコピーします。

In [ ]:
# リサーチフローデータをコピーする

### 2. コピーしたデータを整理する
コピーしたデータを公開するのに適したデータとなるように整理してください。<br>
以下のセルを実行して公開データを表示し、公開に適したデータになるように整理を行ってください。

In [ ]:
# 公開データを表示する

## 公開データにメタデータを付与する
公開データのメタデータ登録を行ってください。<br>
以下のセルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
GakuNin RDMにメタデータを登録する場合はGakuNin RDM上から手動で行ってください。<br>

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※ 再現性を確認する機能は現在開発中です。

### メタデータを検証する
論拠データのメタデータの検証を行ってください<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。<br>

In [ ]:
# 検証するタスクへアクセスするボタンを表示する

## GakuNin RDMに保存する

In [ ]:
# GakuNin RDMに保存する

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する